In [11]:
##(5.2)
# PyTorch Library
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import DataLoader, TensorDataset

# scikit-learn Library
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Pandas Library
import pandas as pd

##(5.3) load wine data
wine = load_wine() 

##(5.4) print data using DataFrame
pd.DataFrame(wine.data, columns=wine.feature_names)

##(5.5) print target 
wine.target

##(5.6) assign 130 of data and target into variable
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

##(5.7) divide dataset: 80% of training set and 20% of test set
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.2)
print(len(train_X))
print(len(test_X))

##(5.8) convert numpy to torch tensor
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()
print(train_X.shape)
print(train_Y.shape)

##(5.9) merge data and target into one tensor
### TensorDataset: Dataset wrapping tensors that have the same size of first dimension.
train = TensorDataset(train_X, train_Y) 
print(train[0])
### DataLoader: set minibatches which are size of 16 and randomly shuffled
train_loader = DataLoader(train, batch_size=16, shuffle=True)

##(5.10) Establish Neural Network 
### class torch.nn.Module: Base class for all NN modules
### Net class inherits nn.Module(parent class)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()   ## utilize __init__ method of parent class
        self.fc1 = nn.Linear(13,96)   ## torch.nn.Linear(in,out,bias=True) 
        self.fc2 = nn.Linear(96, 2)   ##  : applies linear transformation
        
    def forward(self, x):
        x = F.relu(self.fc1(x))       ## Hidden layer: ReLU 
        x = self.fc2(x)               
        return F.log_softmax(x)       ## Output layer: softmax
    
### create instance
model = Net()

##(5.11)
### loss function: cross entropy
criterion = nn.CrossEntropyLoss()

### optimizer: stochastic gradient descent, learning rate 1%
optimizer = optim.SGD(model.parameters(), lr=0.01)

### start learning 
for epoch in range(300):
    total_loss = 0
    ### pop minibatche set out from train_loader
    for train_x, train_y in train_loader:
        ### graph 
        train_x, train_y = Variable(train_x), Variable(train_y)
        ### initialize gradient as PyTorch accumulates gradients
        optimizer.zero_grad()
        ### calc. forward propagation
        output = model(train_x)
        ### calc. loss function
        loss = criterion(output, train_y)
        ### calc. backward propagation
        loss.backward()
        ### update weights
        optimizer.step()
        ### calc. cumulative loss func
        total_loss += loss.data.item()
    if (epoch+1)%50 == 0:
        print(epoch+1, total_loss)
        
        
##(5.12) 
### graph
### torch.autograd.Variable(data): wrapping tensor and logging 
test_x, test_y = Variable(test_X), Variable(test_Y)
### make output 0 or 1
result = torch.max(model(test_x).data, 1)[1]
### accuracy of model
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

accuracy



104
26
torch.Size([104, 13])
torch.Size([104])
(tensor([1.3710e+01, 1.8600e+00, 2.3600e+00, 1.6600e+01, 1.0100e+02, 2.6100e+00,
        2.8800e+00, 2.7000e-01, 1.6900e+00, 3.8000e+00, 1.1100e+00, 4.0000e+00,
        1.0350e+03]), tensor(0))


/Users/seulgi/anaconda3/envs/ptlesson/lib/python3.7/site-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


50 4.813877642154694
100 4.798457860946655
150 4.781480491161346
200 4.814006567001343
250 4.815882444381714
300 4.782549977302551


0.46153846153846156